# Natural Map

## imports

In [1]:
import JordanAlgebra
from sympy import Symbol, Matrix, sqrt, Pow, Add, Rational, postorder_traversal, factor, simplify, nsimplify, latex
from codegen.funcodegen import funcodegen
import numpy as np


## symbols

In [2]:
from JordanAlgebra import epsilon
from friction import mu, rn, rt1, rt2, un, ut1, ut2, xn, xt1, xt2, yn, yt1, yt2, u, r, x, y, ut


## The natural map function

In [3]:
Fnat = y - JordanAlgebra.projection(y - x, epsilon= epsilon*(1+mu))


## Raw jacobians

In [4]:
lambda1, lambda2, u1, u2 = JordanAlgebra.spectral_decomposition(y - x)

#def load(v):

#    with open(v) as r_file:
#        return pickle.load(r_file)

#A = load('Fnat_A')
#B = load('Fnat_B')
#Fnat = load('Fnat')
#Rnow = load('Fnat_Rnow')

In [5]:
A_ = Fnat.jacobian(u)
B_ = Fnat.jacobian(r)

In [6]:
from sympy import Piecewise
def add_split(expr, pos, f1, f2):
    n_expr = expr
    if type(expr) == Add:       
       n_expr1 = Add(*expr.args[:pos])
       n_expr2 = Add(*expr.args[pos:])
       n_expr = Add(f2(n_expr2), f1(n_expr1))
    return n_expr

def add_splitp(p_expr, lpos, f1, f2):
    if hasattr(p_expr, 'is_Piecewise') and p_expr.is_Piecewise:
        return Piecewise(*[(add_split(c[0], lpos[i], f1, f2), c[1]) for i, c in enumerate(p_expr.args)])
    else:
        return add_split(p_expr, lpos[0], f1, f2)

def add_splitv(mat, lpos, f1, f2):
    return Matrix(mat.shape[0], mat.shape[1], 
        lambda i,j: add_splitp(mat[i, j], lpos[i], f1, f2))

def map_matrix(mat, fun, args):
    return Matrix(mat.shape[0], mat.shape[1], 
                  lambda i,j: fun(mat[i, j], *(args[i])))

from sympy import Mul, Add, Pow, Wild
def factor_try(input_expr, fexprs):
    n_expr = input_expr
    subs = dict()   
    for fexpr in fexprs:
        for expr in postorder_traversal(n_expr):
            niexpr = expr   
            for iexpr in subs:
               niexpr = niexpr.subs(iexpr, subs[iexpr])
            expr = niexpr
            if type(expr) == Add:
               fterms = filter(lambda e: type(e)==Mul and fexpr in e.args, expr.args)
               uterms = filter(lambda e: type(e)==Mul and any([fexpr == ee.args[0] for ee in filter(lambda e: type(e)==Pow, e.args)]), expr.args)              
               pterms = filter(lambda e: type(e)==Pow and e.args[1].is_Integer and fexpr == e.args[0], expr.args)
               tterms = filter(lambda e: e == fexpr, expr.args)
               oterms = list(set(expr.args).difference(set(uterms)).difference(set(pterms)).difference(set(tterms)).difference(set(fterms)))
               nfterms = list()
               for e in fterms:
                   nargs = list(e.args)
                   nargs.remove(fexpr)
                   nfterms.append(e.func(*nargs))
               for e in uterms:
                   ne = 1
                   wuterms = filter(lambda ie: type(ie)==Pow and ie.args[0]==fexpr, e.args)
                   for ee in wuterms:
                       ne = Mul(ne, Pow(ee.args[0], ee.args[1]-1))
                   for ee in set(e.args).difference(wuterms):
                       ne = Mul(ne, ee)
                   nfterms.append(ne)
               for e in pterms:
                   nfterms.append(e.func(e.args[0], e.args[1]-1))
               for e in tterms:
                   nfterms.append(1)
               
               nfresult = fexpr * Add(*nfterms)
              
               n_expr = n_expr.subs(expr, Add(Add(*oterms), nfresult))
               subs[expr] = Add(Add(*oterms), nfresult)
    return n_expr
       


In [7]:
from sympy import Wild
expr = sqrt(rt1)
y = Wild('y')
expr.replace(Pow(y, Rational(3,2)), lambda y: y+1)

sqrt(rt1)

In [8]:
factor_try(sqrt(2)*ut1*rt1**4+rt1*ut1+sqrt(2)*rt1*ut2, [sqrt(2), rt1])

rt1*(ut1 + sqrt(2)*(rt1**3*ut1 + ut2))

In [9]:
factor_try(sqrt(rt1+ut1*rt1**7)+ ut1*rt1 + ut2*rt1*rt1, [rt1,2])

rt1*(rt1*ut2 + ut1) + sqrt(rt1*(rt1**6*ut1 + 1))

In [10]:
add_splitp(Piecewise((1 + ut1**2/2 + ut2**2/2, ut1>0), (ut1, ut1<=0)), [1,0], factor, factor)

#map_matrix(Matrix([1 + ut1**2/2 + ut2**2/2]), add_split, [1, factor, factor])

Piecewise(((ut1**2 + ut2**2)/2 + 1, ut1 > 0), (ut1, ut1 <= 0))

ndelta, sqrt((mu*ut2-rt2)**2+(mu*ut1-rt1)**2)

In [11]:
from sympy import init_printing
init_printing(use_latex=True)

from sympy import Symbol
nut = Symbol(r'\lVert {u_{t}} \rVert', negative=False)
nrt = Symbol(r'\lVert {r_{t}} \rVert', negative=False)
lbd1 = Symbol(r'\lambda_1')
lbd2 = Symbol(r'\lambda_2')
#lbd10 = Symbol(r'\lambda_{1|_{\lVert {u_{t}} \rVert=0}}')
lbd10 = Symbol(r'\lambda_{1}^{\prime}')
lbd20 = Symbol(r'\lambda_{2}^{\prime}')
lbd100 = Symbol(r'\lambda_{1}^{\prime\prime}')
lbd200 = Symbol(r'\lambda_{2}^{\prime\prime}')
delta1 = Symbol(r'\Delta_1')
delta2 = Symbol(r'\Delta_2')
ndelta = Symbol(r'\lVert {\Delta} \rVert', negative=False)
lepsilon = Symbol(r'\epsilon')
lut1 = Symbol(r'u_{t1}')
lut2 = Symbol(r'u_{t2}')
lun = Symbol(r'u_n')
lrt1 = Symbol(r'r_{t1}')
lrt2 = Symbol(r'r_{t2}')
lrn = Symbol(r'r_n')

from codegen.funcodegen import flatten_piecewise

lambda1_0 = lambda1.subs(ut1, 0).subs(ut2, 0)
lambda2_0 = lambda2.subs(ut1, 0).subs(ut2, 0)

lambda1_00 = lambda1_0.subs(rt1, 0).subs(rt2, 0)
lambda2_00 = lambda2_0.subs(rt1, 0).subs(rt2, 0)

from codegen.maple import to_sympy
def fix_from_maple(expr):
    
    n_expr = to_sympy(expr)
    try:
        r = n_expr.subs('rt1', rt1).subs('rt2', rt2).subs('rn', rn).subs('ut1', ut1).subs('ut2', ut2).subs('un', un).subs('mu', mu)
        return r
    except:
        print "Warning: subs failed for {0}!!!!".format(n_expr)
        return n_expr

def make_readable(expr, deep=False):
    return flatten_piecewise(expr).subs(lrt1, rt1).subs(lrt2, rt2).subs(lrn, rn).subs(lut1, ut1).subs(lut2, ut2).subs(lun, un).subs(nut, sqrt(ut1**2+ut2**2)).subs(nrt, sqrt(rt1**2+rt2**2)).subs('rt1', rt1).subs('rt2', rt2).subs('rn', rn).subs('ut1', ut1).subs('ut2', ut2).subs('un', un).subs('mu', mu).subs(lambda1, lbd1).subs(lambda2, lbd2).subs(lambda1_0, lbd10).subs(lambda2_0, lbd20).subs(lambda1_00, lbd100).subs(lambda2_00, lbd200).subs(ut1**2+ut2**2,nut**2).subs(rt1**2+rt2**2, nrt**2).subs(mu*ut2-rt2, delta2).subs(mu*ut1-rt1, delta1).subs(delta1**2+delta2**2, ndelta**2).subs(rt1, lrt1).subs(rt2, lrt2).subs(rn, lrn).subs(ut1, lut1).subs(ut2, lut2).subs(un, lun).subs(epsilon, lepsilon).subs(0.5, Rational(1, 2)).subs(sqrt(nrt**2), nrt).subs(sqrt(nut**2), nut).subs(sqrt(ndelta**2), ndelta).subs(sqrt(sqrt(nrt**2)), sqrt(nrt))


In [12]:
lambda1, lambda1_0, lambda1_00

⎛            _____________           ___________________________________      
⎜           ╱    2      2           ╱               2                 2       
⎝μ⋅rn - μ⋅╲╱  ut₁  + ut₂   - un - ╲╱  (-μ⋅ut₁ + rt₁)  + (-μ⋅ut₂ + rt₂)  , μ⋅rn

           _____________           ⎞
          ╱    2      2            ⎟
 - un - ╲╱  rt₁  + rt₂  , μ⋅rn - un⎠

In [13]:
lbd100

\lambda_{1}__{\prime\prime}

In [14]:
from sympy import expand
add_splitv(make_readable(Fnat), [[2], [1,0], [1,1]], factor, factor)


⎡                                       Max(0, \lambda_1) + Max(0, \lambda_2) 
⎢                               μ⋅r_n - ───────────────────────────────────── 
⎢                                                         2                   
⎢                                                                             
⎢⎧  \Delta₁⋅(Max(0, \lambda_1) - Max(0, \lambda_2))                           
⎢⎪- ─────────────────────────────────────────────── + r_{t1}  for \lVert {\Del
⎢⎪              2⋅\lVert {\Delta} \rVert                                      
⎢⎨                                                                            
⎢⎪                                                                            
⎢⎪                          r_{t1}                            for \lVert {\Del
⎢⎩                                                                            
⎢                                                                             
⎢⎧  \Delta₂⋅(Max(0, \lambda_1) - Max(0, \lambda_2)) 

In [15]:
P=make_readable(Fnat[1]).args[0][0]



In [16]:
add_split(make_readable(Fnat), [2], factor, factor)

⎡                                          Max(0, \lambda_1)   Max(0, \lambda_
⎢                                  μ⋅r_n - ───────────────── - ───────────────
⎢                                                  2                   2      
⎢                                                                             
⎢⎧  \Delta₁⋅Max(0, \lambda_1)   \Delta₁⋅Max(0, \lambda_2)                     
⎢⎪- ───────────────────────── + ───────────────────────── + r_{t1}  for \lVert
⎢⎪   2⋅\lVert {\Delta} \rVert    2⋅\lVert {\Delta} \rVert                     
⎢⎨                                                                            
⎢⎪                                                                            
⎢⎪                             r_{t1}                               for \lVert
⎢⎩                                                                            
⎢                                                                             
⎢⎧  \Delta₂⋅Max(0, \lambda_1)   \Delta₂⋅Max(0, \lamb

In [17]:
with open('codegen/tex/fnat.tex', 'w') as f:
    f.write(latex(add_splitv(make_readable(Fnat), [[2], [1,0], [1,1]], factor, factor)))

In [18]:
def nsimp(expr):
    n_expr = expr
    for subexpr in postorder_traversal(expr):
        try:
            simp_subexpr = nsimplify(subexpr)
        except:
            simp_subexpr = subexpr
        if subexpr != simp_subexpr:
            n_expr = n_expr.subs(subexpr, simp_subexpr)
    return n_expr

In [19]:
from sympy import Symbol
import imp
import codegen.maple as maple_module
imp.reload(maple_module)
from codegen.maple import Maple, set_maple
maple = Maple(server='bizet.inria.fr')
set_maple(maple)
t = Symbol('t', positive=True, real=True)
maple_assumes = 'mu >= 0 and mu < 1 and rn >=0 and t > 0 and epsilon > 0'
maple.assume(maple_assumes)


No remote temporary directory (option server_tmpdir) specified, using /tmp/ on bizet.inria.fr


In [20]:
from codegen.maple import _mlimit
maple("diff(sin(2*t), t)")

2*cos(2*t)

In [21]:
from sympy import sin
from codegen.maple import mlimit
mlimit(sin(t), t, 0)

0

In [22]:
A=Matrix(3, 3, lambda i, j: 0)

## A[:, 0]

In [23]:
add_splitv(make_readable(A_[:, 0]), [[2], [2,0], [2,0]], factor, factor)

⎡                             Heaviside(\lambda₁) + Heaviside(\lambda₂)       
⎢                             ─────────────────────────────────────────       
⎢                                                 2                           
⎢                                                                             
⎢⎧\Delta₁⋅(Heaviside(\lambda₁) - Heaviside(\lambda₂))                         
⎢⎪───────────────────────────────────────────────────  for \lVert {\Delta} \rV
⎢⎪              2⋅\lVert {\Delta} \rVert                                      
⎢⎨                                                                            
⎢⎪                                                                            
⎢⎪                         0                           for \lVert {\Delta} \rV
⎢⎩                                                                            
⎢                                                                             
⎢⎧\Delta₂⋅(Heaviside(\lambda₁) - Heaviside(\lambda₂)

In [24]:
# the first column is ok
A[:, 0] = A_[:, 0]

In [25]:
with open('codegen/tex/A0_fnat.tex', 'w') as f:
     f.write(latex(add_splitv(make_readable(A[:, 0]), [[2], [2,0], [2,0]], factor, factor)))

## A[1, 1]

In [26]:
factor(make_readable(A_[0, 1]))

μ⋅(\Delta₁⋅\lVert {u_{t}} \rVert⋅Heaviside(\lambda₁) - \Delta₁⋅\lVert {u_{t}} 
──────────────────────────────────────────────────────────────────────────────
                                                                              

\rVert⋅Heaviside(\lambda₂) + \lVert {\Delta} \rVert⋅u_{t1}⋅Heaviside(\lambda₁)
──────────────────────────────────────────────────────────────────────────────
    2⋅\lVert {\Delta} \rVert⋅\lVert {u_{t}} \rVert                            

 + \lVert {\Delta} \rVert⋅u_{t1}⋅Heaviside(\lambda₂))
─────────────────────────────────────────────────────
                                                     

In [27]:
# we must apply the same limits or subsitution for all the jacobian
# unfortunately substituing 0 for both ut1 and ut2 (in the same order) leads to some NaN
from codegen.maple import mlimit
make_readable(nsimp(mlimit(A_[0, 1].subs(ut1, t).subs(ut2, t), t, 0)))

0.25⋅μ⋅(√2⋅\lVert {r_{t}} \rVert⋅Heaviside(\lambda_{1}__{\prime}) + √2⋅\lVert 
──────────────────────────────────────────────────────────────────────────────
                                                                              

{r_{t}} \rVert⋅Heaviside(\lambda_{2}__{\prime}) - 2.0⋅r_{t1}⋅Heaviside(\lambda
──────────────────────────────────────────────────────────────────────────────
                    \lVert {r_{t}} \rVert                                     

_{1}__{\prime}) + 2.0⋅r_{t1}⋅Heaviside(\lambda_{2}__{\prime}))
──────────────────────────────────────────────────────────────
                                                              

pour \Delta_n différent de 0, ||u_t|| -> 0 (==> rt != 0)

deltan
-> 0

In [28]:
from codegen.maple import mlimit
make_readable((nsimp(mlimit(A_[0, 1].subs(ut1, (1+t)*rt1/mu).subs(ut2, (1+t)*rt2/mu), t, 0))))

μ⋅r_{t1}⋅Heaviside(\lambda_{1}__{\prime})
─────────────────────────────────────────
          \lVert {r_{t}} \rVert          

deltan -> 0 && ut -> 0 ==> rt -> 0

In [29]:
make_readable(nsimp(mlimit(A_[0, 1].subs(ut1, t).subs(ut2, t).subs(rt1, t).subs(rt2, 0), t, 0)))


√2⋅μ⋅Heaviside(\lambda_{1}__{\prime\prime})
───────────────────────────────────────────
                     2                     

In [109]:
from sympy import And, Or, Not, sqrt
from codegen.maple import denoms, to_sympy

def xsimplify(expr):
    expr = fix_from_maple(expr)
    return factor_try(simplify(expr), [2, 1/2, sqrt(2), 1/sqrt(2)])

from sympy.core.numbers import NaN, Infinity, ComplexInfinity

def isfinite(x):
    return not(isinstance(x, NaN) or isinstance(x, Infinity) or isinstance(x, ComplexInfinity))

rt_norm = sqrt(rt1**2+rt2**2)
ut_norm = sqrt(ut1**2+ut2**2)
delta_norm = sqrt((-mu*ut1+rt1)**2+(-mu*ut2+rt2)**2)

# ||delta|| -> 0
def denom_fixA(expr):
    return {
           2.0*mu**2 - 2.0*mu +1.0:(None, None),
           2*mu**2 - 2*mu +1:(None, None),
           2*mu**2 - 2.0*mu +1.0:(None, None),
           rt_norm**2: (xsimplify(nsimp(mlimit(expr.subs(ut1, t).subs(ut2, t).subs(rt1, t).subs(rt2, 0), t, 0))), rt_norm**2 <= epsilon),
           ut_norm**2: (xsimplify(nsimp(mlimit(expr.subs(ut1, t).subs(ut2, t), t, 0))), ut_norm**2 <= epsilon),
           delta_norm**2: (xsimplify(nsimp(mlimit(expr.subs(ut1, (1+t*t)*rt1/mu).subs(ut2, (1+t)*rt2/mu), t, 0))), And(delta_norm**2 <= epsilon*(1+mu), ut_norm**2 >= epsilon)) }

def denom_fixB(expr):
    return {
           2*mu**2 - 2*mu +1:(None, None),
           rt_norm**2: (xsimplify(nsimp(mlimit(expr.subs(ut1, (1+t*t)*rt1/mu).subs(ut2, (1+t)*rt2/mu).subs(rt1, t).subs(rt2, 0), t, 0))), rt_norm**2 <= epsilon),
           delta_norm**2: (xsimplify(nsimp(mlimit(expr.subs(ut1, (1+t*t)*rt1/mu).subs(ut2, (1+t)*rt2/mu), t, 0))), delta_norm**2 <= epsilon*(1+mu)) }

similar_cases = { And(delta_norm**2 <= epsilon*(1+mu), rt_norm**2 <= epsilon): And(rt_norm**2 <= epsilon, ut_norm**2 <= epsilon) }

def fix_similar_cases(expr):
    if expr.is_Piecewise:
       cases = []
       conds = dict()
       for e,c in expr.args:
           if c in similar_cases:
              c = similar_cases[c]
           if c not in conds:
              conds[c] = True
              cases.append((e,c))
       return Piecewise(*cases)
    else:
       return expr  

def add_default_case(expr, cases):
    if len(cases) > 0:
       exprs, conds = zip(*cases)
       if expr not in exprs:
          cases.append((expr, (Not(Or(*conds)))))

def fix_c(fixes, expr, cases):

    for edenoms in denoms(expr):
        edenoms = fix_from_maple(edenoms)
        icases = []
        try:
            efix =fixes[edenoms]
        except:
            print edenoms
            raise None
        if not efix[0] is None:
            fix_c(fixes, fix_from_maple(efix[0]), icases)
            if len(icases) > 0:
                add_default_case(efix[0], icases)
                cases.append((Piecewise(*icases), efix[1]))
            else:
                cases.append(efix)

def natfixA(expr):
    cases = []
    fixes = denom_fixA(expr)
    fix_c(fixes, expr, cases)
    add_default_case(expr, cases)
    if len(cases) > 0:
        return fix_similar_cases(flatten_piecewise(Piecewise(*cases).subs(sqrt(2*rt1**2+2*rt2**2), sqrt(2)*rt_norm)))
    else:
        return expr

def natfixB(expr):
    cases = []
    fixes = denom_fixB(expr)
    fix_c(fixes, expr, cases)
    add_default_case(expr, cases)
    if len(cases) > 0:
        return flatten_piecewise(Piecewise(*cases).subs(sqrt(2*rt1**2+2*rt2**2), sqrt(2)*rt_norm))
    else:
        return expr


In [106]:
natfixA(rt1/sqrt(rt1**2+rt2**2))

⎧                         2      2    
⎪       1          for rt₁  + rt₂  ≤ ε
⎪                                     
⎪      rt₁                2      2    
⎨────────────────  for rt₁  + rt₂  > ε
⎪   _____________                     
⎪  ╱    2      2                      
⎪╲╱  rt₁  + rt₂                       
⎩                                     

In [32]:
natfixA(A_[0, 1])

⎧                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                              ⎛                   ⎛                       ___
⎪                              ⎜                   ⎜                      ╱   
⎪                       0.25⋅μ⋅⎝- 2.0⋅rt₁⋅Heaviside⎝μ⋅rn - 1.0⋅un - 1.0⋅╲╱  rt
⎪                       ──────────────────────────────────────────────────────
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                   

In [33]:
A[0, 1] = natfixA(A_[0, 1])

In [34]:
     
make_readable(A[0, 1])

⎧                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪0.25⋅μ⋅(√2⋅\lVert {r_{t}} \rVert⋅Heaviside(\lambda_{1}__{\prime}) + √2⋅\lVert
⎪─────────────────────────────────────────────────────────────────────────────
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎨                                                                             
⎪                                                                             
⎪                                                   

In [35]:
with open('codegen/tex/A01_fnat.tex', 'w') as f:
     f.write(latex(nsimp(make_readable(A[0, 1]))))

## A[0, 2]

In [36]:
make_readable(A_[0,2])

  ⎛        \Delta₂⋅μ                 μ⋅u_{t2}      ⎞                       ⎛  
  ⎜- ────────────────────── - ─────────────────────⎟⋅Heaviside(\lambda₁)   ⎜──
  ⎝  \lVert {\Delta} \rVert   \lVert {u_{t}} \rVert⎠                       ⎝\l
- ────────────────────────────────────────────────────────────────────── - ───
                                    2                                         

    \Delta₂⋅μ                 μ⋅u_{t2}      ⎞                    
──────────────────── - ─────────────────────⎟⋅Heaviside(\lambda₂)
Vert {\Delta} \rVert   \lVert {u_{t}} \rVert⎠                    
─────────────────────────────────────────────────────────────────
                              2                                  

In [37]:

natfixA(A_[0, 2])


⎧                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                              ⎛                   ⎛                       ___
⎪                              ⎜                   ⎜                      ╱   
⎪                       0.25⋅μ⋅⎝- 2.0⋅rt₂⋅Heaviside⎝μ⋅rn - 1.0⋅un - 1.0⋅╲╱  rt
⎪                       ──────────────────────────────────────────────────────
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                   

deltan != 0, ut -> 0

In [38]:
make_readable(nsimp(sqrt(lambda1_0)))

  _______________________
╲╱ \lambda_{1}__{\prime} 

In [39]:
A[0, 2] = natfixA(A_[0, 2])
     
nsimp(make_readable(A[0, 2]))

⎧                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪0.25⋅μ⋅(√2⋅\lVert {r_{t}} \rVert⋅Heaviside(\lambda_{1}__{\prime}) + √2⋅\lVert
⎪─────────────────────────────────────────────────────────────────────────────
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎨                                                                             
⎪                                                                             
⎪                                                   

In [40]:
with open('codegen/tex/A02_fnat.tex', 'w') as f:
     f.write(latex(nsimp(make_readable(A[0, 2]))))

## A[1, 0]

In [41]:
make_readable(A_[1, 0])

⎧\Delta₁⋅Heaviside(\lambda₁)   \Delta₁⋅Heaviside(\lambda₂)                    
⎪─────────────────────────── - ───────────────────────────  for \lVert {\Delta
⎪  2⋅\lVert {\Delta} \rVert      2⋅\lVert {\Delta} \rVert                     
⎨                                                                             
⎪                                                                             
⎪                            0                              for \lVert {\Delta
⎩                                                                             

        2                   
} \rVert  > \epsilon⋅(μ + 1)
                            
                            
        2                   
} \rVert  ≤ \epsilon⋅(μ + 1)
                            

In [42]:
# just a verification
#mlimit(A_[1, 0].subs(ut1, (1+t)*rt1/mu).subs(ut2, (1+t)*rt2/mu), t, 0)

In [43]:
A[1, 0] = A_[1, 0]

In [44]:
make_readable(A[1, 0])

⎧\Delta₁⋅Heaviside(\lambda₁)   \Delta₁⋅Heaviside(\lambda₂)                    
⎪─────────────────────────── - ───────────────────────────  for \lVert {\Delta
⎪  2⋅\lVert {\Delta} \rVert      2⋅\lVert {\Delta} \rVert                     
⎨                                                                             
⎪                                                                             
⎪                            0                              for \lVert {\Delta
⎩                                                                             

        2                   
} \rVert  > \epsilon⋅(μ + 1)
                            
                            
        2                   
} \rVert  ≤ \epsilon⋅(μ + 1)
                            

In [45]:
with open('codegen/tex/A10_fnat.tex', 'w') as f:
     f.write(latex(factor(make_readable(A[1, 0]))))

## A[1, 1]

In [46]:
make_readable(A_[1, 1])

⎧                                                                             
⎪                                                                             
⎪          ⎛        \Delta₁⋅μ                 μ⋅u_{t1}      ⎞                 
⎪  \Delta₁⋅⎜- ────────────────────── - ─────────────────────⎟⋅Heaviside(\lambd
⎪          ⎝  \lVert {\Delta} \rVert   \lVert {u_{t}} \rVert⎠                 
⎨- ───────────────────────────────────────────────────────────────────────────
⎪                             2⋅\lVert {\Delta} \rVert                        
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎩                                                                             

                                                                              
                                                   

In [47]:
A11_raw = flatten_piecewise(A_[1, 1]).args[0][0]
make_readable(A11_raw)

                                                                              
                                                                              
          ⎛        \Delta₁⋅μ                 μ⋅u_{t1}      ⎞                  
  \Delta₁⋅⎜- ────────────────────── - ─────────────────────⎟⋅Heaviside(\lambda
          ⎝  \lVert {\Delta} \rVert   \lVert {u_{t}} \rVert⎠                  
- ────────────────────────────────────────────────────────────────────────────
                             2⋅\lVert {\Delta} \rVert                         

                                                                              
                                                                              
             ⎛      \Delta₁⋅μ                 μ⋅u_{t1}      ⎞                 
₁)   \Delta₁⋅⎜────────────────────── - ─────────────────────⎟⋅Heaviside(\lambd
             ⎝\lVert {\Delta} \rVert   \lVert {u_{t}} \rVert⎠                 
── + ──────────────────────────────────────────────

In [48]:
#mlimit(A_[1, 1].subs(ut1, t).subs(ut2, t), t, 0)
A11_ut0=nsimp(mlimit(flatten_piecewise(A_[1, 1]).args[0][0].subs(ut1, t).subs(ut2, t), t, 0))
A11_ut0rt0=simplify(nsimp(mlimit(flatten_piecewise(A_[1, 1]).args[0][0].subs(ut1, t).subs(ut2, t).subs(rt1, t).subs(rt2, 0), t, 0)))
simplify(make_readable(A11_ut0rt0))

             2                                       
1.0⋅μ⋅(μ - 1) ⋅Heaviside(\lambda_{1}__{\prime\prime})
─────────────────────────────────────────────────────
                       2                             
                    2⋅μ  - 2⋅μ + 1                   

In [49]:
maple.minimize(2*mu*mu-2*mu+1)

1/2

In [50]:
A11_ut0p = simplify(factor_try(A11_ut0.subs('rt1',rt1).subs('rt2',rt2), [rt1, rt1, rt2, rt2, sqrt(2), 2, rt_norm]))
make_readable(A11_ut0p)

       ⎛       ⎛                                                              
0.25⋅μ⋅⎝r_{t1}⋅⎝r_{t1}⋅(2⋅\lVert {r_{t}} \rVert⋅(Heaviside(\lambda_{1}__{\prim
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              

                                                                              
e}) + Heaviside(\lambda_{2}__{\prime})) + √2⋅r_{t1}⋅(-Heaviside(\lambda_{1}__{
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              

                                                         2                    
\prime}) + Heaviside(\lambda_{2}__{\prime}))) + √2⋅r_{t2} ⋅(-Heaviside(\lambda
──────────────────────────────────────────────────

In [51]:
from sympy import Or
A[1, 1] = natfixA(A11_raw)

In [52]:
# ||delta|| -> 0 && ||ut|| -> 0 ==> ||rt|| > 0
make_readable(A[1, 1])

⎧                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪       ⎛                              2                                      
⎪0.25⋅μ⋅⎝2⋅\lVert {r_{t}} \rVert⋅r_{t1} ⋅Heaviside(\lambda_{1}__{\prime}) + 2⋅
⎪─────────────────────────────────────────────────────────────────────────────
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                   

In [53]:
with open('codegen/tex/A11_fnat.tex', 'w') as f:
     f.write(latex(make_readable(A[1, 1])))

## A[1, 2]

In [54]:
make_readable(A_[1, 2])

⎧                                                                             
⎪                                                                            \
⎪\Delta₁⋅\Delta₂⋅μ⋅Max(0, \lambda_1)   \Delta₁⋅\Delta₂⋅μ⋅Max(0, \lambda_2)    
⎪─────────────────────────────────── - ─────────────────────────────────── - ─
⎨                             3                                     3         
⎪     2⋅\lVert {\Delta} \rVert              2⋅\lVert {\Delta} \rVert          
⎪                                                                             
⎪                                                                             
⎩                                                                             

       ⎛        \Delta₂⋅μ                 μ⋅u_{t2}      ⎞                     
Delta₁⋅⎜- ────────────────────── - ─────────────────────⎟⋅Heaviside(\lambda₁) 
       ⎝  \lVert {\Delta} \rVert   \lVert {u_{t}} \rVert⎠                     
───────────────────────────────────────────────────

In [55]:
A12_raw=flatten_piecewise(A_[1, 2]).args[0][0]

In [56]:
A[1, 2] = natfixA(A12_raw)
make_readable(A[1, 2])

⎧                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪              ⎛                                                              
⎪0.25⋅μ⋅r_{t1}⋅⎝2⋅\lVert {r_{t}} \rVert⋅r_{t2}⋅Heaviside(\lambda_{1}__{\prime}
⎪─────────────────────────────────────────────────────────────────────────────
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                   

In [57]:
with open('codegen/tex/A12_fnat.tex', 'w') as f:
     f.write(latex(make_readable(A[1, 2])))

## A[2, 0]

In [58]:
make_readable(A_[2, 0])

⎧\Delta₂⋅Heaviside(\lambda₁)   \Delta₂⋅Heaviside(\lambda₂)                    
⎪─────────────────────────── - ───────────────────────────  for \lVert {\Delta
⎪  2⋅\lVert {\Delta} \rVert      2⋅\lVert {\Delta} \rVert                     
⎨                                                                             
⎪         Heaviside(\lambda₁)   Heaviside(\lambda₂)                           
⎪       - ─────────────────── + ───────────────────         for \lVert {\Delta
⎩                  2                     2                                    

        2                   
} \rVert  > \epsilon⋅(μ + 1)
                            
                            
        2                   
} \rVert  ≤ \epsilon⋅(μ + 1)
                            

## A[2, 1]

In [59]:
make_readable(A_[2, 1])

⎧                                                                             
⎪                                                                            \
⎪\Delta₁⋅\Delta₂⋅μ⋅Max(0, \lambda_1)   \Delta₁⋅\Delta₂⋅μ⋅Max(0, \lambda_2)    
⎪─────────────────────────────────── - ─────────────────────────────────── - ─
⎪                             3                                     3         
⎪     2⋅\lVert {\Delta} \rVert              2⋅\lVert {\Delta} \rVert          
⎨                                                                             
⎪                                              ⎛        \Delta₁⋅μ             
⎪                                              ⎜- ────────────────────── - ───
⎪                                              ⎝  \lVert {\Delta} \rVert   \lV
⎪                                              ───────────────────────────────
⎪                                                                             
⎩                                                   

In [60]:
A21_raw = flatten_piecewise(A_[2, 1]).args[0][0]

In [61]:
A[2, 1] = natfixA(A21_raw)

In [62]:
denoms(A21_raw)

   ⎛⎡   2      2                2                 2⎤⎞
set⎝⎣ut₁  + ut₂ , (-μ⋅ut₁ + rt₁)  + (-μ⋅ut₂ + rt₂) ⎦⎠

In [63]:
make_readable(A[2, 1])

⎧                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪              ⎛                                                              
⎪0.25⋅μ⋅r_{t2}⋅⎝2⋅\lVert {r_{t}} \rVert⋅r_{t1}⋅Heaviside(\lambda_{1}__{\prime}
⎪─────────────────────────────────────────────────────────────────────────────
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                   

In [64]:
with open('codegen/tex/A21_fnat.tex', 'w') as f:
     f.write(latex(make_readable(A[2, 1])))

## A[2, 2]

In [65]:
make_readable(A_[2, 2])

⎧                                                                             
⎪                                                                             
⎪          ⎛        \Delta₂⋅μ                 μ⋅u_{t2}      ⎞                 
⎪  \Delta₂⋅⎜- ────────────────────── - ─────────────────────⎟⋅Heaviside(\lambd
⎪          ⎝  \lVert {\Delta} \rVert   \lVert {u_{t}} \rVert⎠                 
⎪- ───────────────────────────────────────────────────────────────────────────
⎨                             2⋅\lVert {\Delta} \rVert                        
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎩                                                   

In [66]:
A22_raw = flatten_piecewise(A_[2, 2]).args[0][0]

In [67]:
A[2, 2] = natfixA(A22_raw)
make_readable(A[2, 2])

⎧                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪       ⎛                              2                                      
⎪0.25⋅μ⋅⎝2⋅\lVert {r_{t}} \rVert⋅r_{t2} ⋅Heaviside(\lambda_{1}__{\prime}) + 2⋅
⎪─────────────────────────────────────────────────────────────────────────────
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                   

In [68]:
mlimit(simplify(mlimit(A22_raw.subs(ut1, t).subs(ut2, t), t, 0)).subs(rt1, t).subs(rt2, 0), t, 0)

       ⎛                                    ⎛                       __________
       ⎜                     2              ⎜                      ╱    2     
0.25⋅μ⋅⎝- 1.4142135623731⋅rt₁ ⋅rt₂⋅Heaviside⎝μ⋅rn - 1.0⋅un - 1.0⋅╲╱  rt₁  + rt
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              
                                                                              

___⎞                                     ⎛                   _____________⎞   
 2 ⎟                      2              ⎜                  ╱    2      2 ⎟   
₂  ⎠ + 1.4142135623731⋅rt₁ ⋅rt₂⋅Heaviside⎝μ⋅rn - 1.0⋅un + ╲╱  rt₁  + rt₂  ⎠ - 
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                   

In [69]:
with open('codegen/tex/A22_fnat.tex', 'w') as f:
     f.write(latex(make_readable(A[2, 2])))

# B

In [70]:
B = Matrix(3, 3, lambda i,j: 0)

## B[:, 0]

In [71]:
B[:, 0] = B_[:, 0]
make_readable(B[:, 0])

⎡                                μ⋅Heaviside(\lambda₁)   μ⋅Heaviside(\lambda₂)
⎢                              - ───────────────────── - ─────────────────────
⎢                                          2                       2          
⎢                                                                             
⎢⎧  \Delta₁⋅μ⋅Heaviside(\lambda₁)   \Delta₁⋅μ⋅Heaviside(\lambda₂)             
⎢⎪- ───────────────────────────── + ─────────────────────────────  for \lVert 
⎢⎪     2⋅\lVert {\Delta} \rVert        2⋅\lVert {\Delta} \rVert               
⎢⎨                                                                            
⎢⎪                                                                            
⎢⎪                               0                                 for \lVert 
⎢⎩                                                                            
⎢                                                                             
⎢⎧  \Delta₂⋅μ⋅Heaviside(\lambda₁)   \Delta₂⋅μ⋅Heavis

In [72]:
with open('codegen/tex/B0_fnat.tex', 'w') as f:
     f.write(latex(make_readable(B[:, 0])))

## B[0, 1]

In [73]:
make_readable(B_[0, 1])


  \Delta₁⋅Heaviside(\lambda₁)   \Delta₁⋅Heaviside(\lambda₂)
- ─────────────────────────── + ───────────────────────────
    2⋅\lVert {\Delta} \rVert      2⋅\lVert {\Delta} \rVert 

In [74]:
B[0, 1] = natfixB(B_[0, 1])

In [75]:
make_readable(B[0, 1])

⎧                                                                             
⎪                            0.0                              for \lVert {\Del
⎪                                                                             
⎨  \Delta₁⋅Heaviside(\lambda₁)   \Delta₁⋅Heaviside(\lambda₂)                  
⎪- ─────────────────────────── + ───────────────────────────  for \lVert {\Del
⎪    2⋅\lVert {\Delta} \rVert      2⋅\lVert {\Delta} \rVert                   
⎩                                                                             

          2                   
ta} \rVert  ≤ \epsilon⋅(μ + 1)
                              
          2                   
ta} \rVert  > \epsilon⋅(μ + 1)
                              
                              

In [76]:
with open('codegen/tex/B01_fnat.tex', 'w') as f:
     f.write(latex(make_readable(B[0, 1])))

## B[0, 2]

In [77]:
make_readable(B_[0, 2])

  \Delta₂⋅Heaviside(\lambda₁)   \Delta₂⋅Heaviside(\lambda₂)
- ─────────────────────────── + ───────────────────────────
    2⋅\lVert {\Delta} \rVert      2⋅\lVert {\Delta} \rVert 

In [78]:
B[0, 2] = natfixB(B_[0, 2])

In [79]:
make_readable(B[0, 2])

⎧                                                                             
⎪                            0.0                              for \lVert {\Del
⎪                                                                             
⎨  \Delta₂⋅Heaviside(\lambda₁)   \Delta₂⋅Heaviside(\lambda₂)                  
⎪- ─────────────────────────── + ───────────────────────────  for \lVert {\Del
⎪    2⋅\lVert {\Delta} \rVert      2⋅\lVert {\Delta} \rVert                   
⎩                                                                             

          2                   
ta} \rVert  ≤ \epsilon⋅(μ + 1)
                              
          2                   
ta} \rVert  > \epsilon⋅(μ + 1)
                              
                              

In [80]:
with open('codegen/tex/B02_fnat.tex', 'w') as f:
     f.write(latex(make_readable(B[0, 2])))

## B[1, 1]

In [81]:
make_readable(B_[1, 1])

⎧                                                                ⎛            
⎪                                                                ⎜          \D
⎪                                                                ⎜- ──────────
⎪         2                              2                       ⎜            
⎪  \Delta₁ ⋅Heaviside(\lambda₁)   \Delta₁ ⋅Heaviside(\lambda₂)   ⎝  \lVert {\D
⎨- ──────────────────────────── - ──────────────────────────── - ─────────────
⎪                           2                              2                  
⎪   2⋅\lVert {\Delta} \rVert       2⋅\lVert {\Delta} \rVert                   
⎪                                                                             
⎪                                                                             
⎩                                                                             

     2                                ⎞                     ⎛               2 
elta₁                     1           ⎟            

In [82]:
B[1, 1] = B_[1, 1]

In [83]:
with open('codegen/tex/B11_fnat.tex', 'w') as f:
     f.write(latex(make_readable(B[1, 1])))


## B[1, 2]

In [84]:
make_readable(B_[1, 2])

⎧  \Delta₁⋅\Delta₂⋅Heaviside(\lambda₁)   \Delta₁⋅\Delta₂⋅Heaviside(\lambda₂)  
⎪- ─────────────────────────────────── - ─────────────────────────────────── -
⎪                               2                                     2       
⎨       2⋅\lVert {\Delta} \rVert              2⋅\lVert {\Delta} \rVert        
⎪                                                                             
⎪                                                                             
⎩                                                                         0   

 \Delta₁⋅\Delta₂⋅Max(0, \lambda_1)   \Delta₁⋅\Delta₂⋅Max(0, \lambda_2)        
 ───────────────────────────────── + ─────────────────────────────────  for \l
                             3                                   3            
     2⋅\lVert {\Delta} \rVert            2⋅\lVert {\Delta} \rVert             
                                                                              
                                                   

In [85]:
B[1, 2] = B_[1, 2]

In [86]:
with open('codegen/tex/B12_fnat.tex', 'w') as f:
     f.write(latex(make_readable(B[1, 2])))

## B[2, 1]

In [87]:
B21_delta0 = flatten_piecewise(B_[2, 1]).args[0][0]
make_readable(B21_delta0)

  \Delta₁⋅\Delta₂⋅Heaviside(\lambda₁)   \Delta₁⋅\Delta₂⋅Heaviside(\lambda₂)   
- ─────────────────────────────────── - ─────────────────────────────────── - 
                               2                                     2        
       2⋅\lVert {\Delta} \rVert              2⋅\lVert {\Delta} \rVert         

\Delta₁⋅\Delta₂⋅Max(0, \lambda_1)   \Delta₁⋅\Delta₂⋅Max(0, \lambda_2)
───────────────────────────────── + ─────────────────────────────────
                            3                                   3    
    2⋅\lVert {\Delta} \rVert            2⋅\lVert {\Delta} \rVert     

In [88]:
make_readable(mlimit(B21_delta0.subs(ut1, (1+t)*rt1/mu).subs(ut2, (1+t)*rt2/mu), t, 0))


-1.0⋅r_{t1}⋅r_{t2}⋅Heaviside(\lambda_{1}__{\prime}) 
────────────────────────────────────────────────────
                                    2               
               \lVert {r_{t}} \rVert                

In [89]:
B21_delta1 = flatten_piecewise(B_[2, 1]).args[1][0]
make_readable(B21_delta1)

\Delta₁⋅Heaviside(\lambda₁)   \Delta₁⋅Heaviside(\lambda₂)
─────────────────────────── + ───────────────────────────
  2⋅\lVert {\Delta} \rVert      2⋅\lVert {\Delta} \rVert 

In [90]:
make_readable(mlimit(B21_delta1.subs(ut1, (1+t)*rt1/mu).subs(ut2, (1+t)*rt2/mu), t, 0))


r_{t1}⋅Heaviside(\lambda_{1}__{\prime})
───────────────────────────────────────
         \lVert {r_{t}} \rVert         

In [110]:
make_readable(natfixB(B21_delta0))

⎧                                                                             
⎪                                                                        0.0  
⎪                                                                             
⎪                                               -1.0⋅r_{t1}⋅r_{t2}⋅Heaviside(\
⎪                                               ──────────────────────────────
⎪                                                                             
⎨                                                              \lVert {r_{t}} 
⎪                                                                             
⎪  \Delta₁⋅\Delta₂⋅Heaviside(\lambda₁)   \Delta₁⋅\Delta₂⋅Heaviside(\lambda₂)  
⎪- ─────────────────────────────────── - ─────────────────────────────────── -
⎪                               2                                     2       
⎪       2⋅\lVert {\Delta} \rVert              2⋅\lVert {\Delta} \rVert        
⎩                                                   

In [111]:
B[2, 1] = natfixB(B21_delta0)



In [112]:
with open('codegen/tex/B21_fnat.tex', 'w') as f:
     f.write(latex(make_readable(B[2, 1])))

## B[2, 2]

In [113]:
make_readable(B_[2, 2])

⎧                                                                ⎛            
⎪                                                                ⎜          \D
⎪                                                                ⎜- ──────────
⎪         2                              2                       ⎜            
⎪  \Delta₂ ⋅Heaviside(\lambda₁)   \Delta₂ ⋅Heaviside(\lambda₂)   ⎝  \lVert {\D
⎪- ──────────────────────────── - ──────────────────────────── - ─────────────
⎨                           2                              2                  
⎪   2⋅\lVert {\Delta} \rVert       2⋅\lVert {\Delta} \rVert                   
⎪                                                                             
⎪                                                                          \De
⎪                                                                          ───
⎪                                                                            2
⎩                                                   

In [114]:
B22_delta0 = flatten_piecewise(B_[2, 2]).args[0][0]
B22_delta1 = flatten_piecewise(B_[2, 2]).args[1][0]

In [161]:
from codegen.maple import _to_sympy
nsimp(mlimit(simplify(simplify(B22_delta0).subs(ut1, (1+t*t)*rt1/mu).subs(ut2, (1+t)*rt2/mu).subs(rt1, t).subs(rt2, t)), t, 0))
#make_readable(mlimit((B22_delta0).subs(ut1, (1+t)*rt1/mu).subs(ut2, (1+t)*rt2/mu), t, 0))

-1.0⋅Heaviside(μ⋅rn - 1.0⋅un) + 1.0

In [183]:
from sympy import diff
X=Symbol('X', real=True)
Y=Symbol('Y', real=True)
F=sqrt(X*X+Y*Y)
mlimit(diff(F, X).subs(X, sin(t*t)).subs(Y, t), t, 0), mlimit(diff(F, Y).subs(X, sin(t*t)).subs(Y, t), t, 0)

(0, 1)

In [0]:
make_readable(B[2, 2])


In [0]:
with open('codegen/tex/B22_fnat.tex', 'w') as f:
     f.write(latex(make_readable(B[2, 2])))

# Code generation

In [0]:
import pickle
with open('Fnat_A', 'w') as A_file:
    pickle.dump(A, A_file)

with open('Fnat_B', 'w') as B_file:
    pickle.dump(B, B_file)

with open('Fnat', 'w') as Fnat_file:
    pickle.dump(Fnat, Fnat_file)

    
    

## partial code

In [0]:
import pickle
def load(v):

    with open(v) as r_file:
        return pickle.load(r_file)

A = load('Fnat_A')
B = load('Fnat_B')
Fnat = load('Fnat')

In [0]:
from sympy import var, Interval
intervals = {mu: Interval(0, 1), rn: Interval(0, 1e6)}

In [0]:
from codegen.maple import denoms_bounds
print set(denoms(A)), set(bounds(A, intervals=intervals))

In [0]:
EPSILON = np.finfo(float).eps

var('rhon rhot1 rhot2')

for i in range(3):
    for j in range(3):  
        print i, j
        print funcodegen('tA{0}{1}'.format(i, j), A[i, j].subs(epsilon, EPSILON),
                         variables=[rn, rt1, rt2, un, ut1, ut2, mu, rhon, rhot1,
                                                 rhot2],
                     intervals=intervals,
                     array_format='Fortran',
                     epsilon_inf=EPSILON,
                     epsilon_power=3,
                     with_files_generation=True,
                     assertions=True, contracts=False, main_check=True)

None

In [0]:
%%bash
framac_path=`frama-c -print-path`
echo $framac_path
#frama-c -kernel-msg-key pp -cpp-extra-args="$framac_path/libc -I$framac_path -std=c99" -pp-annot -print -ocode t_temp.c -then -val tA00.c -slevel 100 -then -wp  tA00.c -wp-model nat+real -no-print   -wp-timeout 40 -wp-prover alt-ergo -wp-alt-ergo-opt="-backward-compat" -wp-out temp -wp-verbose 2  -then -wp -wp-prover cvc3 -then -wp -wp-prover z3 -then -wp -wp-prover zenon -then -report 

## full code generation

In [0]:
from sympy import var, Interval
intervals = {mu: Interval(0, 1), rn: Interval(0, 1e6)}

function_name = 'NaturalMap'
target_name = 'FAB'

EPSILON = np.finfo(float).eps

expression = {
    'fc3d_{0}FABGenerated'.format(function_name): Fnat.row_join(A).row_join(B),
    'fc3d_{0}FGenerated'.format(function_name):   Fnat,
    'fc3d_{0}ABGenerated'.format(function_name):  A.row_join(B)}

C_function_name = 'fc3d_{0}{1}Generated'.format(
    function_name, target_name)

var('rhon rhot1 rhot2')

print funcodegen(C_function_name, expression[C_function_name].subs(epsilon, EPSILON),
                 variables=[rn, rt1, rt2, un, ut1, ut2, mu, rhon, rhot1,
                            rhot2],
                 intervals=intervals,
                 array_format='Fortran',
                 epsilon_inf=EPSILON,
                 epsilon_power=3,
                 with_files_generation=True,
                 assertions=True, contracts=False, main_check=True)


In [0]:
maple.solve(0.2*t*t - 0.2*t + 0.025)